In [1]:
import re
# Numpy 
import numpy as np
# Pickle
import pickle
# Pandas
import pandas as pd
# Hugging Face
import huggingface_hub
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
# PyTorch
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# SkLearn
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
# nltk.download()
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


False

In [2]:
# Load datasets 
# Hate Xplain
hate_xplain = pd.read_csv("data/hate_xplain.csv")

# Implicit Hate 
implicit_hate = pd.read_csv('data/implicit-hate-corpus/implicit_hate_v1_stg2_posts.tsv', delimiter='\t')
label_map = {
    'white_grievance': 0, 'incitement': 1, 'inferiority': 2,
    'irony': 3, 'stereotypical': 4, 'threatening': 5, 'other': 6
}

implicit_hate['class_label'] = implicit_hate['implicit_class'].map(label_map)
implicit_hate.drop("extra_implicit_class", axis=1, inplace=True)

# Toxic-Spans
annotations = pd.read_csv('data/toxic-spans/annotations.csv')
comments = pd.read_csv('data/toxic-spans/comments.csv')

toxic_spans = pd.merge(annotations, comments, on='comment_id')

In [3]:
bert = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(bert)
bert_model = DistilBertForSequenceClassification.from_pretrained(bert)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def tokenize_data(texts, labels, tokenizer, max_length=512):
    if isinstance(texts, pd.Series):
        texts = texts.tolist()
    texts = [str(text) for text in texts] 

    if isinstance(labels, pd.Series):
        labels = labels.tolist()
    labels = torch.tensor(labels, dtype=torch.long)
    
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    dataset = torch.utils.data.TensorDataset(encodings["input_ids"], encodings["attention_mask"], labels)
    return dataset


In [13]:
def train(model, train_loader, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        for batch in train_loader: 
            input_ids, attn_mask, labels = [b for b in batch]
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attn_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

def evaluate(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in test_loader: 
            input_ids, attn_mask, labels = [b for b in batch]

            outputs = model(input_ids, attention_mask=attn_mask)
            preds = torch.argmax(outputs.logits, dim=1)

            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())

    report = classification_report(all_labels, all_preds, target_names=["Hate Speech", "Normal", "Offensive"], output_dict=True)
    overall_acc = np.mean(np.array(all_preds) == np.array(all_labels))

    return report, overall_acc
            

In [14]:
hx_dataset = tokenize_data(hate_xplain['tweet'], hate_xplain['class'], tokenizer)
hx_train, hx_test = train_test_split(hx_dataset, test_size=0.2, random_state=42)

hx_train_loader = torch.utils.data.DataLoader(hx_train, batch_size=16, shuffle=True)
hx_test_loader = torch.utils.data.DataLoader(hx_test, batch_size=16, shuffle=False)

In [15]:
optimizer = optim.AdamW(bert_model.parameters(), lr=5e-5)
train(bert_model, hx_train, optimizer, 3)

RuntimeError: The size of tensor a (481) must match the size of tensor b (512) at non-singleton dimension 1